In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

pd.set_option("display.max_columns", 100)
sns.set_theme()
start_time = datetime.datetime.now()

In [2]:
# Define Data Paths
### Cross Border Wait Times
xbwt_loc = "C:/Users/rohan.sirupa/OneDrive - Resource Systems Group, Inc/SANDAG/2022_waittime/inputs/border_wait_times_2022.csv"

# Read Data
xbwt_df = pd.read_csv(xbwt_loc)

### Process Data
xbwt_df['date_reformat'] = pd.to_datetime(xbwt_df['date'])
xbwt_df['weekday'] = xbwt_df['date_reformat'].dt.day_name()

In [3]:
xbwt_df['port_name'].value_counts()

San Ysidro                        11884
Calexico West                     11008
Otay Mesa Passenger                9636
Calexico East                      8808
Tecate                             3439
Andrade                            2950
Otay Mesa Commercial               2355
Otay Mesa Cross Border Express     1369
San Ysidro PedWest                    1
Name: port_name, dtype: int64

In [4]:
xbwt_df['description'].value_counts()

Pedestrian - Standard           14092
Passenger Vehicle - Standard    12767
Passenger Vehicle - SENTRI       8515
Passenger Vehicle - Ready        6961
Pedestrian - Ready               4939
Commercial - Standard            3030
Commercial - FAST                1146
Name: description, dtype: int64

In [5]:
### Rename description
description_dict = {
    'Pedestrian - Standard': 'PedestrianWait',
    'Pedestrian - Ready': 'PedestrianWait',
    'Passenger Vehicle - Standard': 'StandardWait',
    'Passenger Vehicle - SENTRI': 'SENTRIWait',
    'Passenger Vehicle - Ready': 'ReadyWait',
    'Commercial - Standard': 'Com_StandardWait',
    'Commercial - FAST': 'Com_FastWait'
}
xbwt_df['description'] = xbwt_df['description'].replace(description_dict)

In [6]:
### Identify weekdays
xbwt_df['is_weekday'] = 0
xbwt_df.loc[xbwt_df['weekday'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']), 'is_weekday'] = 1

### Define POE list
poe_list = ['San Ysidro', 'Otay Mesa Passenger', 'Tecate']
poe_dict = {
    'San Ysidro': 0,
    'Otay Mesa Passenger': 1,
    'Tecate': 2
}

# Identify data to keep
xbwt_df['keep'] = 0
### Only want data from March and first half of November
xbwt_df.loc[xbwt_df['month']==3, 'keep'] = 1
xbwt_df.loc[(xbwt_df['month']==11) & (xbwt_df['day']<=15), 'keep'] = 1
### Remove weekends
xbwt_df.loc[xbwt_df['is_weekday']==0, 'keep'] = 0
### Remove POE not in [San Ysidro, Otay Mesa, Tecate]
xbwt_df.loc[~xbwt_df['port_name'].isin(poe_list), 'keep'] = 0
xbwt_df['poe'] = xbwt_df['port_name'].replace(poe_dict)
xbwt_df.keep.value_counts()

0    46425
1     5025
Name: keep, dtype: int64

In [7]:
### Data to use
xbwt_filter_df = xbwt_df[xbwt_df['keep']==1]
xbwt_filter_df

,date,date_id,yr,month,day,hour_id,port_id,port_name,crossing_type,description,lanes_open,delay,date_reformat,weekday,is_weekday,keep,poe
20110,2022-03-01 00:00:00,20220301,2022,3,1,0,250401,San Ysidro,1,StandardWait,2,50,2022-03-01 00:00:00,Tuesday,1,1,0
20111,2022-03-01 00:00:00,20220301,2022,3,1,0,250401,San Ysidro,4,ReadyWait,11,30,2022-03-01 00:00:00,Tuesday,1,1,0
20112,2022-03-01 00:00:00,20220301,2022,3,1,0,250401,San Ysidro,7,SENTRIWait,1,5,2022-03-01 00:00:00,Tuesday,1,1,0
20113,2022-03-01 00:00:00,20220301,2022,3,1,0,250401,San Ysidro,10,PedestrianWait,3,5,2022-03-01 00:00:00,Tuesday,1,1,0
20114,2022-03-01 00:00:00,20220301,2022,3,1,0,250401,San Ysidro,13,PedestrianWait,3,5,2022-03-01 00:00:00,Tuesday,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34222,2022-11-15 22:00:00,20221115,2022,11,15,22,250601,Otay Mesa Passenger,4,ReadyWait,2,45,2022-11-15 22:00:00,Tuesday,1,1,1
34223,2022-11-15 22:00:00,20221115,2022,11,15,22,250601,Otay Mesa Passenger,10,PedestrianWait,5,0,2022-11-15 22:00:00,Tuesday,1,1,1
34228,2022-11-15 23:00:00,20221115,2022,11,15,23,250601,Otay Mesa Passenger,1,StandardWait,1,25,2022-11-15 23:00:00,Tuesday,1,1,1
34229,2022-11-15 23:00:00,20221115,2022,11,15,23,250601,Otay Mesa Passenger,4,ReadyWait,2,35,2022-11-15 23:00:00,Tuesday,1,1,1


In [8]:
### Time Period Dictionary
tp_dict = {
    1: 45,
    2: 47,
    3: 1,
    4: 3,
    5: 5,
    6: 7,
    7: 9,
    8: 11,
    9: 13,
    10: 15,
    11: 17,
    12: 19,
    13: 21,
    14: 23,
    15: 25,
    16: 27,
    17: 29,
    18: 31,
    19: 33,
    20: 35,
    21: 37,
    22: 39,
    23: 41,
    0: 43
}

### Summary Table
summary_df = pd.pivot_table(xbwt_filter_df, values='delay', index=['poe', 'hour_id'], columns='description', aggfunc=np.mean).fillna(0).reset_index()
summary_df['StartPeriod'] = summary_df['hour_id'].replace(tp_dict)
summary_df['EndPeriod'] = summary_df['StartPeriod'] + 1
summary_df.rename({'hour_id': 'StartHour'}, axis=1, inplace=True)
summary_df['EndHour'] = summary_df['StartHour'] + 1
summary_df = summary_df[['poe', 'StartHour', 'EndHour', 'StartPeriod', 'EndPeriod', 'Com_StandardWait', 'PedestrianWait', 'ReadyWait', 'SENTRIWait', 'StandardWait']]


### Fill-in missing start hours
for i in [0, 1, 2]:
    if len(summary_df[summary_df['poe']==i].StartHour.unique()) != 24:
        for j in range(0, 24):
            if j not in summary_df[summary_df['poe']==2].StartHour.unique():
                summary_df.loc[len(summary_df.index)] = [i, j, j + 1, tp_dict[j], tp_dict[j] + 1, 999, 999, 999, 999, 999]

summary_df['EndHour'] = summary_df['EndHour'].replace({24: 0})
summary_df = summary_df.sort_values(by = ['poe', 'StartHour'])

### Frequency Table
summary_freq_df = pd.pivot_table(xbwt_filter_df, values='delay', index=['poe', 'hour_id'], columns='description', aggfunc='count').fillna(0).reset_index()
summary_freq_df.rename({'hour_id': 'StartHour'}, axis=1, inplace=True)

### Fill-in missing start hours
for i in [0, 1, 2]:
    if len(summary_freq_df[summary_freq_df['poe']==i].StartHour.unique()) != 24:
        for j in range(0, 24):
            if j not in summary_freq_df[summary_freq_df['poe']==2].StartHour.unique():
                summary_freq_df.loc[len(summary_freq_df.index)] = [i, j, 0, 0, 0, 0, 0]

summary_freq_df.rename({'Com_StandardWait': 'Com_StandardWait_Freq', 
                        'PedestrianWait': 'PedestrianWait_Freq', 
                        'ReadyWait': 'ReadyWait_Freq', 
                        'SENTRIWait': 'SENTRIWait_Freq', 
                        'StandardWait': 'StandardWait_Freq'}, axis=1, inplace=True)


### Merge summary and frequency tables
summary_df = pd.merge(left=summary_df, right=summary_freq_df, on=['poe', 'StartHour'], how='left')
summary_df.drop(['Com_StandardWait', 'Com_StandardWait_Freq'], axis=1, inplace=True)
summary_df

description,poe,StartHour,EndHour,StartPeriod,EndPeriod,PedestrianWait,ReadyWait,SENTRIWait,StandardWait,PedestrianWait_Freq,ReadyWait_Freq,SENTRIWait_Freq,StandardWait_Freq
0,0,0,1,43,44,4.772727,47.045455,7.272727,59.545455,44.0,22.0,22.0,22.0
1,0,1,2,45,46,5.238095,47.857143,6.428571,61.904762,42.0,21.0,21.0,21.0
2,0,2,3,47,48,4.880952,53.809524,6.428571,68.571429,42.0,21.0,21.0,21.0
3,0,3,4,1,2,5.543478,62.608696,10.000000,77.391304,46.0,23.0,23.0,23.0
4,0,4,5,3,4,8.382353,77.058824,15.294118,90.294118,34.0,17.0,17.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2,19,20,33,34,2.818182,0.000000,0.000000,27.272727,11.0,0.0,0.0,11.0
68,2,20,21,35,36,2.529412,0.000000,0.000000,18.823529,17.0,0.0,0.0,17.0
69,2,21,22,37,38,2.444444,0.000000,0.000000,14.117647,18.0,0.0,0.0,17.0
70,2,22,23,39,40,999.000000,999.000000,999.000000,999.000000,0.0,0.0,0.0,0.0


In [9]:
### Save the cross-border wait times summary to CSV
summary_df.to_csv("C:/Users/rohan.sirupa/OneDrive - Resource Systems Group, Inc/SANDAG/2022_waittime/inputs/crossborder_avg_waittimes_2022.csv", index=False)

In [10]:
end_time = datetime.datetime.now()
print("Start Time:", start_time)
print("End Time:", end_time)
print("Run Time:", round(end_time.timestamp()-start_time.timestamp(), 3), "sec")

Start Time: 2023-05-08 09:41:04.516844
End Time: 2023-05-08 09:41:06.561288
Run Time: 2.044 sec
